In [1]:
#librerías para analizar datos:
import os, numpy as np, pandas as pd
import scipy # herramientas y algoritmos matemáticos
import matplotlib.pyplot as plt, seaborn as sns # gráficos
import os
import math
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt # Visualizaciones
import seaborn as sns # Visualizaciones mas sencillas

# Limpieza data V5

In [2]:
df = pd.read_csv("dataV5transpuesto.csv", sep="|",low_memory=False)
df.head(2)

,ID,PERIODO_CODIGO,PERIODO_NOMBRE,ESTUDIANTE_CARRERA_CODIGO,PROGRAMA_NOMBRE_LARGO,ESTUDIANTE_CARRERA_MALLA_COD,PROGRAMA_MODALIDAD_NOMBRE,PROGRAMA_NIVEL_NOMBRE,CREDITOS_TOTAL,CURSOS,...,Tipo de centro de rehabilitación social,Tipo de colegio,Tipo de discapacidad que posee,Tipo de documento legal de ingresos del núcleo familiar,Tipo de mérito,Tipo de parentesco,Tipo de representación estudiantil,Titulación aprobada por la Senescyt,Usted o algún miembro de su familia cuenta con registro social,Usted o algún miembro de su familia recibe el bono de desarrollo humano
0,32,202264,OCT/2022 - FEB/2023,PSIC_D1,PSICOLOGIA,PSD6,DISTANCIA,GRADO,15,5,...,NaN,NaN,NaN,Declaración Juramentada,NaN,NaN,NaN,NaN,NaN,NaN
1,32,202264,OCT/2022 - FEB/2023,PSIC_D1,PSICOLOGIA,PSD6,DISTANCIA,GRADO,15,5,...,NaN,NaN,NaN,Declaración Juramentada,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(581184, 90)

In [4]:
# pd.set_option('display.max_rows', None)  # Muestra todas las filas

# data.isna().sum()

## Corregir columnas con valores en null

In [5]:
df['Disciplina deportiva en la que participa'] = df['Disciplina deportiva en la que participa'].fillna('Ninguno')
df['Fué declarado usted abanderado del pabellón nacional'] = df['Fué declarado usted abanderado del pabellón nacional'].fillna('No')
df['Nombre de club universitario'] = df['Nombre de club universitario'].fillna('Ninguno')
df['Nombre de la federación deportiva'] = df['Nombre de la federación deportiva'].fillna('Ninguno')
df['Nombre del grupo de arte de UTPL al que pertenece'] = df['Nombre del grupo de arte de UTPL al que pertenece'].fillna('Ninguno')
df['Cuáles son los ingresos familiares'] = df['Cuáles son los ingresos familiares'].fillna('0')
df['Porcentaje de discapacidad'] = df['Porcentaje de discapacidad'].fillna(0)
df['Número de miembros del núcleo familiar (1°consanguinidad)'] = df['Número de miembros del núcleo familiar (1°consanguinidad)'].fillna(0)
df['Qué miembro de su núcleo familiar registra la calamidad doméstica'] = df['Qué miembro de su núcleo familiar registra la calamidad doméstica'].fillna('Ninguno')
df['Servicios médicos con los que cuenta la familia'] = df['Servicios médicos con los que cuenta la familia'].fillna('Ninguno')
df['Su núcleo familiar cuenta con alguna calamidad doméstica'] = df['Su núcleo familiar cuenta con alguna calamidad doméstica'].fillna('Ninguno')
df['Tipo de discapacidad que posee'] = df['Tipo de discapacidad que posee'].fillna('Ninguna')
df['Tipo de parentesco'] = df['Tipo de parentesco'].fillna('No Definido')
df['Los ingresos provienen de'] = df['Los ingresos provienen de'].fillna('Desconocido')
df['Monto de ingresos nominales familiares'] = df['Monto de ingresos nominales familiares'].fillna(0)


In [6]:
df['BECAS'].unique()

array([0])

## Eliminar espacios vacios antes y despues de los valores del dataframe

In [7]:
# Eliminar espacios en blanco al inicio y final de los nombres de las columnas
df.columns = df.columns.str.strip()

# Eliminar espacios en blanco al inicio y final de los valores en cada columna de tipo string
for column in df.select_dtypes(include='object').columns:
    df[column] = df[column].str.strip()


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581184 entries, 0 to 581183
Data columns (total 90 columns):
 #   Column                                                                     Non-Null Count   Dtype  
---  ------                                                                     --------------   -----  
 0   ID                                                                         581184 non-null  int64  
 1   PERIODO_CODIGO                                                             581184 non-null  int64  
 2   PERIODO_NOMBRE                                                             581184 non-null  object 
 3   ESTUDIANTE_CARRERA_CODIGO                                                  581184 non-null  object 
 4   PROGRAMA_NOMBRE_LARGO                                                      581184 non-null  object 
 5   ESTUDIANTE_CARRERA_MALLA_COD                                               581184 non-null  object 
 6   PROGRAMA_MODALIDAD_NOMBRE                   

## Conversión de formatos

In [9]:
# Convertir columnas a int
df['BECA_PORCENTAJE'] = df['BECA_PORCENTAJE'].astype(int)
df['Número de miembros del núcleo familiar (1°consanguinidad)'] = df['Número de miembros del núcleo familiar (1°consanguinidad)'].replace(',', '.', regex=False).astype(float)

In [10]:

df['Su núcleo familiar cuenta con alguna calamidad doméstica'].unique()

array(['Ninguno', 'Enfermedad catastrófica', 'Defunción',
       'Familiar con discapacidad'], dtype=object)

## Silenciar columnas

In [11]:
# Filtrar solo las columnas que contienen "_COD" en su nombre
columns_with_cod = df.filter(like='_CODIGO').columns.tolist() + \
             df.filter(like='CODIGO_').columns.tolist() + \
             df.filter(like='_COD').columns.tolist() 

# Calcular el porcentaje de valores nulos por columna
porcentajeNulosDataCat = (df.isnull().sum() / len(df)) * 100

# Filtrar las columnas con más del 50% de valores nulos
columnas_a_eliminar_nulos = porcentajeNulosDataCat[porcentajeNulosDataCat > 50].index.tolist()  # Asegúrate que sea una lista


columnasNoConsideradas= ['CREDITOS_TOTAL','CURSOS_ANULADOS','CURSOS_ELIMINADOS',
            'CONVENIOS','Alguien de su núcleo familiar tiene RUC',
            'Alguien de su núcleo familiar tiene RUC o RISE',
            'Año de aceptación del cupo por la Senescyt',
            'Carnet emitido por','Fecha de aviso de salida del IESS (DD-MM-AA)',
           'Modalidad aprobada por la Senescyt','Nombre de la congregación religiosa a la que pertenece',
            'Nombre del centro de rehabilitación social','Nombre del colegio donde se graduó',
            'Parentesco con el familiar que es o fué héroe o heroína',
            'Perfil','Provincia a la que pertenece la congregación religiosa',
            'Qué miembro de su núcleo familiar cuenta con el bono de desarrollo humano',
            'Tiene algún familiar que es o fué héroe o heroína',
            'Tiene aviso de salida del IESS','Tiene contrato indefinido en UTPL',
            'Tipo de afiliación al IESS del solicitante',
            'Tipo de centro de rehabilitación social','Tipo de representación estudiantil',
            'Titulación aprobada por la Senescyt',
            'Usted o algún miembro de su familia cuenta con registro social',
            'Usted o algún miembro de su familia recibe el bono de desarrollo humano',
            'Rol que desempeña en UTPL','En su núcleo familiar alguien tiene calamidad doméstica','Tiempo en el grupo',
           'Tipo de documento legal de ingresos del núcleo familiar','Tipo de mérito','BECAS','DESCUENTOS','BECA_DETALLE_DESCRIPCION',
           'Cuántos miembros son en su familia (1°consanguinidad)','Cuántos miembros son en su familia(1°consanguinidad y afinidad)',
            'Año de graduación','Año que quedó desempleado','Nombre de la dependencia a la que pertenece',
            'Nombre de la unidad educativa en la que obtuvo el mérito académico',
            'Nombre de la universidad a la que pertenece','Nombre de la universidad de intercambio',
            'Nombre del proyecto a realizar','Nombre del pueblo o nacionalidad indígena a la que pertenece',
            'Promedio de graduación','Seleccione la región de su pueblo o nacionalidad indígena',
            'Tiempo en el club universitario','Tipo de Beca de Responsabilidad Universitaria','Tipo de colegio',
           'SISTEMA_EVALUACION','CRN','Nombre del reconocimiento','Qué miembro de su núcleo familiar registra la calamidad doméstica',
            'PARALELO','Nombre de club universitario','Nombre de la federación deportiva',
            'Nombre del grupo de arte de UTPL al que pertenece','Tipo de parentesco','CURSO_NOMBRE'
                        ]
# Convertir la lista 'columnasNoConsideradas' en un DataFrame vacío para poder concatenarlo (opcional)
df_columnas_no_consideradas = pd.DataFrame(columns=columnasNoConsideradas)

# Crear un nuevo DataFrame dfSilence con las columnas seleccionadas
dfSilence = pd.concat([df[columns_with_cod], df[columnasNoConsideradas], df[columnas_a_eliminar_nulos]], axis=1)

# Eliminar las columnas del DataFrame original que han sido identificadas
data1 = df.drop(columns=columns_with_cod + columnas_a_eliminar_nulos + columnasNoConsideradas, errors='ignore')
# Mostrar el DataFrame modificado
#dfTraspuesto2.head(10)

### Correción de tipeo en disciplinas deportivas

In [12]:
data1['Disciplina deportiva en la que participa'] = data1['Disciplina deportiva en la que participa'].str.title()
# data1['Disciplina deportiva en la que participa'].unique()

# Reemplazar los valores incorrectos con los valores correctos
data1['Disciplina deportiva en la que participa'] = data1['Disciplina deportiva en la que participa'].replace({
    'Fútbol': 'Futbol',
    'ajedrez': 'Ajedrez',
    'escalada': 'Escalada',
    'baloncesto': 'Baloncesto',
    'Basket': 'Baloncesto',
    'Basquetbol': 'Baloncesto'   
})

# Verificar que los cambios se realizaron
data1['Disciplina deportiva en la que participa'].unique()

array(['Ninguno', 'Wushu', 'Ciclismo', 'Futbol', 'Ajedrez', 'Taekwondo',
       'Escalada', 'Mtb', 'Patinaje V', 'P. Velocid', 'Pesista', 'Judo',
       'Atletismo', 'Baloncesto', 'Tenis De M', 'Natación', 'Karate',
       'Pumse', 'Natacion'], dtype=object)

In [13]:
pd.set_option('display.max_row', None)

data1.isna().sum()

ID                                                           0
PERIODO_NOMBRE                                               0
PROGRAMA_NOMBRE_LARGO                                        0
PROGRAMA_MODALIDAD_NOMBRE                                    0
PROGRAMA_NIVEL_NOMBRE                                        0
CURSOS                                                       0
CURSOS_RETIRADOS                                             0
BECA_NOMBRE                                                  0
BECA_PORCENTAJE                                              0
FINAL_NOTA                                                   0
TOTALFIN_NOTA                                                0
ESTADO_CALIFICACION                                          0
BIM2_NOTA                                                    0
BIM1_NOTA                                                    0
Cuáles son los ingresos familiares                           0
Disciplina deportiva en la que participa               

# GUARDAR Nuevo dataframe V2 limpio Con valores catego2ricos

In [14]:
# Guardar el DataFrame resultante en un archivo CSV
data1.to_csv('dataCleanV5.csv', index=False)

In [15]:
data = pd.read_csv("dataCleanV5.csv", sep=",",low_memory=False)
data.head(2)

,ID,PERIODO_NOMBRE,PROGRAMA_NOMBRE_LARGO,PROGRAMA_MODALIDAD_NOMBRE,PROGRAMA_NIVEL_NOMBRE,CURSOS,CURSOS_RETIRADOS,BECA_NOMBRE,BECA_PORCENTAJE,FINAL_NOTA,...,Cuáles son los ingresos familiares,Disciplina deportiva en la que participa,Fué declarado usted abanderado del pabellón nacional,Los ingresos provienen de,Monto de ingresos nominales familiares,Número de miembros del núcleo familiar (1°consanguinidad),Porcentaje de discapacidad,Servicios médicos con los que cuenta la familia,Su núcleo familiar cuenta con alguna calamidad doméstica,Tipo de discapacidad que posee
0,32,OCT/2022 - FEB/2023,PSICOLOGIA,DISTANCIA,GRADO,5,0,BECA NIVEL DE INGRESOS,40,9.81,...,0,Ninguno,No,Desconocido,0.0,0.0,0.0,Ninguno,Ninguno,Ninguna
1,32,OCT/2022 - FEB/2023,PSICOLOGIA,DISTANCIA,GRADO,5,0,BECA NIVEL DE INGRESOS,40,9.59,...,0,Ninguno,No,Desconocido,0.0,0.0,0.0,Ninguno,Ninguno,Ninguna


In [16]:
# data.info()

# Reemplazzar valores categóricos por valores numéricos representativos. DUMMIES


### Columna de ingresos familiares 


Discretización de valores ategóricos a numéricos  con el promedio de los dos valores mismos

In [17]:

# Asegúrate de que todos los valores sean cadenas de texto
data['Cuáles son los ingresos familiares'] = data['Cuáles son los ingresos familiares'].astype(str)

# Limpia los espacios alrededor del guion
data['Cuáles son los ingresos familiares'] = data['Cuáles son los ingresos familiares'].str.replace(r'\s*-\s*', ' - ', regex=True)

# Diccionario de rangos a valores promedio
ingresos_map = {
    '0': 0,  # Para el caso de '0'
    '0 - 500': 250,
    '501 - 1000': 750,
    '1001 - 1500': 1250,
    '1501 - 2000': 1750,
    '2001 - 3000': 2500,
    'Mayor a 3000': 3500
}

# Reemplaza los valores en la columna
data['ingresosPromedio'] = data['Cuáles son los ingresos familiares'].map(ingresos_map)

# Convierte la nueva columna a float
data['ingresosPromedio'] = data['ingresosPromedio'].astype(int)



In [18]:
data['ingresosPromedio'].unique()

array([   0,  250, 1250,  750, 1750, 2500, 3500])


### Asigna un número entero a cada categoría  
- **Los ingresos provienen de**

In [19]:
#  columna original antes de la transformación
data['Los ingresos provienen de_original'] = data['Los ingresos provienen de']

# Aplica el LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['fuenteIngresos'] = le.fit_transform(data['Los ingresos provienen de'])

# Mostrar los valores únicos de la nueva columna codificada
print(data['Los ingresos provienen de_original'].unique())

print(data['fuenteIngresos'].unique())


['Desconocido' 'Negocio propio formal' 'Relación de dependencia'
 'Otros ingresos (arriendos. pensiones. ayudas familiares)'
 'Negocio propio informal' 'Por ayuda de los padres']
[0 1 5 3 2 4]


# Agrupar becas por tipo Ingreso, Social y Mérito 

In [20]:
# Mapeo para agrupar las becas en categorías
becas_map = {
    'BECA NIVEL DE INGRESOS': 'Ingreso',
    'BECA DESEMPLEO': 'Ingreso',
    'BECA FUNCIONARIOS': 'Ingreso',
    'BECA PUEBLOS Y NACIONALIDADES DEL ECUADOR': 'Ingreso',
    'BECA DISCAPACIDAD': 'Social',
    'BECA FAMILIAR ESTUDIANTES': 'Social',
    'BECA HIJOS DE HEROES Y HEROINAS': 'Social',
    'BECA FAMILIAR HIJO DE DOCENTE': 'Social',
    'BECA FAMILIAR ADMINISTRATIVO Y REPRESENTANTE DEL CONSEJO SUPERIOR GRADO': 'Social',
    'BECA DEPORTISTAS DESTACADOS': 'Mérito',
    'BECA CULTURAL': 'Mérito',
    'BECA MERITOS UNIVERSITARIOS': 'Mérito',
    'BECA CLUB UNIVERSITARIO': 'Mérito',
    'BECA ABANDERADOS': 'Mérito',
    'BECA BACHILLERES DESTACADOS': 'Mérito',
    'BECA POLITICA DE CUOTAS': 'Social',
    'BECA RESPONSABILIDAD UNIVERSITARIA GRADO': 'Social',
    'BECA REPRESENTACION ESTUDIANTIL GRADO': 'Social',
    'BECA MOVILIDAD ESTUDIANTIL GRADO': 'Social',
    'BECA DIOCESIS DEL PAIS (OTRAS TITULACIONES)': 'Social',
    'BECA DIOCESIS TITULACIONES: CIENCIAS HUMANAS Y RELIGIOSAS/ RELIGIÓN': 'Social',
    'BECA DIOCESIS DE LOJA (OTRAS TITULACIONES)': 'Social',
    'BECA CENTROS DE REHABILITACION SOCIAL (Privados de la libertad)': 'Social'
}

# Aplicar el mapeo
data['tipo_beca'] = data['BECA_NOMBRE'].map(becas_map)

# Verificar los resultados para ver si hay algún NaN
print(data['tipo_beca'].unique())

# Aplicar pd.get_dummies para convertir la columna 'tipo_beca' en columnas binarias
data = pd.get_dummies(data, columns=['tipo_beca'], prefix='BECA', drop_first=False)

# Verificar las columnas generadas
# print(data.columns)


['Ingreso' 'Social' 'Mérito']


In [21]:
# Aplicar pd.get_dummies para convertir la columna 'Tipo de discapacidad que posee' en columnas binarias
data = pd.get_dummies(data, columns=['Tipo de discapacidad que posee'], prefix='dis', drop_first=False)

# Verificar las columnas generadas
print(data.columns)


Index(['ID', 'PERIODO_NOMBRE', 'PROGRAMA_NOMBRE_LARGO',
       'PROGRAMA_MODALIDAD_NOMBRE', 'PROGRAMA_NIVEL_NOMBRE', 'CURSOS',
       'CURSOS_RETIRADOS', 'BECA_NOMBRE', 'BECA_PORCENTAJE', 'FINAL_NOTA',
       'TOTALFIN_NOTA', 'ESTADO_CALIFICACION', 'BIM2_NOTA', 'BIM1_NOTA',
       'Cuáles son los ingresos familiares',
       'Disciplina deportiva en la que participa',
       'Fué declarado usted abanderado del pabellón nacional',
       'Los ingresos provienen de', 'Monto de ingresos nominales familiares',
       'Número de miembros del núcleo familiar (1°consanguinidad)',
       'Porcentaje de discapacidad',
       'Servicios médicos con los que cuenta la familia',
       'Su núcleo familiar cuenta con alguna calamidad doméstica',
       'ingresosPromedio', 'Los ingresos provienen de_original',
       'fuenteIngresos', 'BECA_Ingreso', 'BECA_Mérito', 'BECA_Social',
       'dis_Auditiva', 'dis_Física', 'dis_Intelectual', 'dis_Lenguaje',
       'dis_Multidiscapacidades', 'dis_Ninguna', '

In [22]:
# '_Beca Ingreso','_Beca Mérito','_Beca Social'

In [23]:
# Crear una nueva columna con 1 para los que tienen un deporte y 0 para los que tienen "Ninguno"
data['participa_en_deporte'] = data['Disciplina deportiva en la que participa'].apply(lambda x: 1 if x != 'Ninguno' else 0)

# Verificar los resultados
print(data['participa_en_deporte'].unique())

[0 1]



## Conversión de valores

In [24]:
# Crear una nueva columna con 1 para los que tienen un deporte y 0 para los que tienen "Ninguno"
data['calamidadDomestica'] = data['Su núcleo familiar cuenta con alguna calamidad doméstica'].apply(lambda x: 1 if x != 'Ninguno' else 0)

# Verificar los resultados
print(data['calamidadDomestica'].unique())

[0 1]


## Conversión de valores binarios

In [25]:
data = data.replace({'SI': 1, 'NO': 0, 'Si': 1, 'No': 0})
data = data.infer_objects(copy=False)

/var/folders/3n/wr348_g54y3dnpq24d5zdn9w0000gn/T/ipykernel_2937/1715377763.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({'SI': 1, 'NO': 0, 'Si': 1, 'No': 0})


### Aprobado o reprobado a valores binarios

In [26]:
data['ESTADO_CALIFICACION'] = data['ESTADO_CALIFICACION'].apply(lambda x: 1 if x == 'Aprobado' else 0)


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581184 entries, 0 to 581183
Data columns (total 39 columns):
 #   Column                                                     Non-Null Count   Dtype  
---  ------                                                     --------------   -----  
 0   ID                                                         581184 non-null  int64  
 1   PERIODO_NOMBRE                                             581184 non-null  object 
 2   PROGRAMA_NOMBRE_LARGO                                      581184 non-null  object 
 3   PROGRAMA_MODALIDAD_NOMBRE                                  581184 non-null  object 
 4   PROGRAMA_NIVEL_NOMBRE                                      581184 non-null  object 
 5   CURSOS                                                     581184 non-null  int64  
 6   CURSOS_RETIRADOS                                           581184 non-null  int64  
 7   BECA_NOMBRE                                                581184 non-null  object 

### 1. ESTUDIANTE_CARRERA_NIVEL

In [28]:
# O convertir a binario
data['PROGRAMA_NIVEL_NOMBRE'] = data['PROGRAMA_NIVEL_NOMBRE'].map({'GRADO': 0, 'TECNICO TECNOLOGICO': 1})


### 2. ESTUDIANTE_CARRERA_MODALIDAD

In [29]:
# Convertir a tipo categórico
data['PROGRAMA_MODALIDAD_NOMBRE'] = data['PROGRAMA_MODALIDAD_NOMBRE'].map({'DISTANCIA': 0, 'PRESENCIAL': 1})


## 3. Tratamiento de la columna de carreras 

In [30]:
# Contar las frecuencias de cada carrera
frecuencias = data['PROGRAMA_NOMBRE_LARGO'].value_counts()

# Seleccionar un umbral para las carreras más frecuentes (e.g., las 3 más comunes)
umbral = 3
carreras_frecuentes = frecuencias.nlargest(umbral).index
# print("Carreras s:", dataUnido['ESTUDIANTE_CARRERA_NOMBRE'].value_counts())
print("Carreras más frecuentes:", carreras_frecuentes)


Carreras más frecuentes: Index(['DERECHO', 'PSICOLOGIA', 'ARQUITECTURA'], dtype='object', name='PROGRAMA_NOMBRE_LARGO')


In [31]:
# Reemplazar las carreras menos frecuentes con "OTRAS"
data['PROGRAMA_NOMBRE_LARGO'] = data['PROGRAMA_NOMBRE_LARGO'].apply(
    lambda x: x if x in carreras_frecuentes else 'OTRAS'
)

# Verificar las categorías resultantes
print(data['PROGRAMA_NOMBRE_LARGO'].unique())


['PSICOLOGIA' 'DERECHO' 'ARQUITECTURA' 'OTRAS']


#### Aplicar One-Hot Encoding a la columna agrupada: de carreras


In [32]:
# Crear variables dummies para las carreras
data = pd.get_dummies(data, columns=['PROGRAMA_NOMBRE_LARGO'], prefix='CARRERA')


## 4. Tratamiento de la columna de años PERIODO NOMBRE 

In [33]:
# Extraer los años de inicio y fin como listas
data['periodoAñoInicio'] = data['PERIODO_NOMBRE'].str.extract(r'(\d{4})', expand=False)
data['periodoAñoFin'] = data['PERIODO_NOMBRE'].str.extract(r'(\d{4})$', expand=False)

# Convertir las columnas de años a enteros y manejar valores nulos
data['periodoAñoInicio'] = data['periodoAñoInicio'].fillna(0).astype(int)
data['periodoAñoFin'] = data['periodoAñoFin'].fillna(0).astype(int)

# Crear la columna 'periodoAños' con los años dentro del rango
data['periodoAños'] = data.apply(
    lambda row: list(range(row['periodoAñoInicio'], row['periodoAñoFin'] + 1)),
    axis=1
)

# Explode para separar los años en filas individuales
data = data.explode('periodoAños')

# Asegurarte de que la columna 'periodoAños' sea de tipo entero
data['periodoAños'] = data['periodoAños'].astype(int)

# Verificar la nueva columna con los años individuales y su tipo
print(data['periodoAños'].head())
# print(data_expanded['periodoAños'].dtype)


0    2022
0    2023
1    2022
1    2023
2    2022
Name: periodoAños, dtype: int64


### 5. Servicios médicos con los que cuenta la familia a binarios

In [34]:
# Crear una nueva columna con 1 para los que tienen un deporte y 0 para los que tienen "Ninguno"
data['servicioMedico'] = data['Servicios médicos con los que cuenta la familia'].apply(lambda x: 1 if x != 'Ninguno' else 0)

# Verificar los resultados
print(data['servicioMedico'].unique())

[0 1]


# Eliminar variables transformadas a numérico

In [35]:
# Eliminar las columnas especificadas
data1 = data.drop(columns=['PERIODO_NOMBRE','BECA_NOMBRE','Cuáles son los ingresos familiares','Disciplina deportiva en la que participa','Los ingresos provienen de','Su núcleo familiar cuenta con alguna calamidad doméstica','Servicios médicos con los que cuenta la familia','Los ingresos provienen de_original'])


In [36]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 926102 entries, 0 to 581183
Data columns (total 38 columns):
 #   Column                                                     Non-Null Count   Dtype  
---  ------                                                     --------------   -----  
 0   ID                                                         926102 non-null  int64  
 1   PROGRAMA_MODALIDAD_NOMBRE                                  926102 non-null  int64  
 2   PROGRAMA_NIVEL_NOMBRE                                      926102 non-null  int64  
 3   CURSOS                                                     926102 non-null  int64  
 4   CURSOS_RETIRADOS                                           926102 non-null  int64  
 5   BECA_PORCENTAJE                                            926102 non-null  int64  
 6   FINAL_NOTA                                                 926102 non-null  float64
 7   TOTALFIN_NOTA                                              926102 non-null  float64
 8  

In [37]:
data1.head()

,ID,PROGRAMA_MODALIDAD_NOMBRE,PROGRAMA_NIVEL_NOMBRE,CURSOS,CURSOS_RETIRADOS,BECA_PORCENTAJE,FINAL_NOTA,TOTALFIN_NOTA,ESTADO_CALIFICACION,BIM2_NOTA,...,participa_en_deporte,calamidadDomestica,CARRERA_ARQUITECTURA,CARRERA_DERECHO,CARRERA_OTRAS,CARRERA_PSICOLOGIA,periodoAñoInicio,periodoAñoFin,periodoAños,servicioMedico
0,32,0,0,5,0,40,9.81,9.81,1,9.67,...,0,0,False,False,False,True,2022,2023,2022,0
0,32,0,0,5,0,40,9.81,9.81,1,9.67,...,0,0,False,False,False,True,2022,2023,2023,0
1,32,0,0,5,0,40,9.59,9.59,1,9.45,...,0,0,False,False,False,True,2022,2023,2022,0
1,32,0,0,5,0,40,9.59,9.59,1,9.45,...,0,0,False,False,False,True,2022,2023,2023,0
2,32,0,0,5,0,40,9.06,9.06,1,9.11,...,0,0,False,False,False,True,2022,2023,2022,0


In [38]:
# Filtrar las columnas de tipo objeto (categóricas)
obj_columns = data1.select_dtypes(include=['object'])

# Mostrar los valores únicos de cada columna de tipo objeto
for col in obj_columns.columns:
    print(f"Valores únicos de la columna {col}:")
    print(obj_columns[col].unique())
    print("\n")


In [43]:
dataUnido = data1.astype({col: 'int' for col in data.select_dtypes(include=['bool']).columns})

# Guardar el DataFrame resultante en un archivo CSV
dataUnido.to_csv('dataDummiesV5.csv', index=False, sep=";")


In [44]:
dataUnido.info()

<class 'pandas.core.frame.DataFrame'>
Index: 926102 entries, 0 to 581183
Data columns (total 38 columns):
 #   Column                                                     Non-Null Count   Dtype  
---  ------                                                     --------------   -----  
 0   ID                                                         926102 non-null  int64  
 1   PROGRAMA_MODALIDAD_NOMBRE                                  926102 non-null  int64  
 2   PROGRAMA_NIVEL_NOMBRE                                      926102 non-null  int64  
 3   CURSOS                                                     926102 non-null  int64  
 4   CURSOS_RETIRADOS                                           926102 non-null  int64  
 5   BECA_PORCENTAJE                                            926102 non-null  int64  
 6   FINAL_NOTA                                                 926102 non-null  float64
 7   TOTALFIN_NOTA                                              926102 non-null  float64
 8  